# Topic Modeling and Natural Language Processing with Twitter Data

##  Jason Anastasopoulos
##  December 4, 2018
### Email: ljanastas@uga.edu

The code below provides a brief introduction on acquiring Twitter data using the twitter API via Python. For this exercise I will be acquiring Donald Trump's tweets and will try to figure out what the topics his tweets are using the Latent Dirichlet Allocation  Topic Model.

In [9]:
import csv
import os,re,csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim import corpora, models
import gensim
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import twitter
import json

Here we enter our Twitter credentials. These can be acquired through 

In [10]:
api = twitter.Api(consumer_key='4NHHnFAx6AwbhipaG0aGJ2P16',
                      consumer_secret='RKhXZANu7SnXKrtHUw5K276jJbg5Y8i3tDyn2c9YmcVNpQBevc',
                      access_token_key='18249358-tUB1Pg2HDDJtIKAwanfMYk379yFsDLOTLEQBPTaLG',
                      access_token_secret='zoaYBorszhARknoaEsfT5l7pDogvTSINKOV0a0EboMFNk')

print(api.VerifyCredentials())

AttributeError: module 'twitter' has no attribute 'Api'

Search the Twitter API using keywords.

In [47]:
search = api.GetSearch("Runoff Georgia") # Replace happy with your search
for tweet in search:
    print(tweet.id, tweet.text)
    
len(search)

(1070159200641302529, u'RT @USAElectionInfo: Republican Brad Raffensperger has defeated Democrat John Barrow in the runoff election for Georgia Secretary of State.')
(1070159162208710656, u'RT @Greg_Palast: For example... Rahiem Shabazz, who found himself on the #Georgia purge list we made public, is still not listed as registe\u2026')
(1070159129317134336, u"RT @alexsalvinews: #BREAKING on @OANN: Republican Brad Raffensperger has defeated former Democratic congressman John Barrow in Georgia's ru\u2026")
(1070159107196416001, u'RT @voxdotcom: Republican Brad Raffensperger has won the race for Georgia\u2019s next secretary of state.\n\nHe\u2019ll be responsible for responding t\u2026')
(1070159090452639745, u"RT @alexsalvinews: #BREAKING on @OANN: Republican Brad Raffensperger has defeated former Democratic congressman John Barrow in Georgia's ru\u2026")
(1070159076053667840, u'RT @nytimes: Georgia secretary of state live election results https://t.co/5OPkvSAcyF')
(1070159057862955008, 

15

The Python twitter library has a lot of cool functions that you can use and learn about through the help() function

In [48]:
help(api.GetUserTimeline)


Help on method GetUserTimeline in module twitter.api:

GetUserTimeline(self, user_id=None, screen_name=None, since_id=None, max_id=None, count=None, include_rts=True, trim_user=False, exclude_replies=False) method of twitter.api.Api instance
    Fetch the sequence of public Status messages for a single user.
    
    The twitter.Api instance must be authenticated if the user is private.
    
    Args:
      user_id (int, optional):
        Specifies the ID of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid user ID
        is also a valid screen name.
      screen_name (str, optional):
        Specifies the screen name of the user for whom to return the
        user_timeline. Helpful for disambiguating when a valid screen
        name is also a user ID.
      since_id (int, optional):
        Returns results with an ID greater than (that is, more recent
        than) the specified ID. There are limits to the number of
        Tweets which c

In [51]:
t = api.GetUserTimeline(screen_name="realDonaldTrump", count=5000)
tweets = [i.AsDict() for i in t]

tweettext = [i["text"] for i in tweets]

len(tweettext)

tweettext[0:10]

[u'.....China does not want Tariffs!',
 u'We are either going to have a REAL DEAL with China, or no deal at all - at which point we will be charging major Ta\u2026 https://t.co/JIqNBYOxT8',
 u'....in the world. I want clean air and clean water and have been making great strides in improving America\u2019s enviro\u2026 https://t.co/iQ4aLjD0Kb',
 u'I am glad that my friend @EmmanuelMacron and the protestors in Paris have agreed with the conclusion I reached two\u2026 https://t.co/YcxKsnDGwt',
 u'RT @charliekirk11: There are riots in socialist France because of radical leftist fuel taxes\n\nMedia barely mentioning this\n\nAmerica is boom\u2026',
 u'Could somebody please explain to the Democrats (we need their votes) that our Country losses 250 Billion Dollars a\u2026 https://t.co/Z8nKReskdg',
 u'.....But if a fair deal is able to be made with China, one that does all of the many things we know must be finally\u2026 https://t.co/VMfAQjVXmz',
 u'....I am a Tariff Man. When people or countri

## Cleaning the text

In [28]:
######## So far so good not lets clean this up ###
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
texts = []

Here we are pre-processing the text by creating a tokenizer that splits the documents up into tokens (words or phrases), creating a dictionary of stop words and creating a "stemmer" which stems the words (ie removing "-ing" endings etc. We also remove extraneous "bill related" words such as "propXX_XXXX".

In [54]:
for i in tweettext:
    #print "Processing",i
    # clean and tokenize document string
    tokens = tokenizer.tokenize(i)
    # remove all numbers
    tokens = [x for x in tokens if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    # remove structural words
    tokens = [x for x in tokens if len(x) > 1]
    tokens = [x.lower() for x in tokens]
    tokens = [x for x in tokens if 'http' not in x]
    tokens = [x for x in tokens if x not in "_"]
    tokens = [x for x in tokens if x not in 'rt']
    tokens = [x for x in tokens if x not in ".co"]
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    texts.append(stemmed_tokens)

dictionaryall = corpora.Dictionary(texts)

corpusall = [dictionaryall.doc2bow(text) for text in texts]

texts[1]



[u'either',
 u'go',
 u'real',
 u'deal',
 u'china',
 u'deal',
 u'point',
 u'will',
 u'charg',
 u'major',
 u'ta',
 u'co',
 u'jiqnbyoxt8']

In [39]:
tweettext[0:10]

[u'.....China does not want Tariffs!',
 u'We are either going to have a REAL DEAL with China, or no deal at all - at which point we will be charging major Ta\u2026 https://t.co/JIqNBYOxT8',
 u'....in the world. I want clean air and clean water and have been making great strides in improving America\u2019s enviro\u2026 https://t.co/iQ4aLjD0Kb',
 u'I am glad that my friend @EmmanuelMacron and the protestors in Paris have agreed with the conclusion I reached two\u2026 https://t.co/YcxKsnDGwt',
 u'RT @charliekirk11: There are riots in socialist France because of radical leftist fuel taxes\n\nMedia barely mentioning this\n\nAmerica is boom\u2026',
 u'Could somebody please explain to the Democrats (we need their votes) that our Country losses 250 Billion Dollars a\u2026 https://t.co/Z8nKReskdg',
 u'.....But if a fair deal is able to be made with China, one that does all of the many things we know must be finally\u2026 https://t.co/VMfAQjVXmz',
 u'....I am a Tariff Man. When people or countri

This code performs tokenization, stop word removal and number removal and places the corpora into a clean list that will be ready for analysis using the Latent Dirichlet Allocation. 

## Estimating the Latent Dirichlet Allocation model

In [55]:
# generate LDA model
ldamodelall = gensim.models.ldamodel.LdaModel(corpusall, num_topics=5, 
                                              id2word = dictionaryall, passes=20,
                                              minimum_probability=0)



The code above estimates a 5 topic topic model using Trump's tweets

In [56]:
# First 25 are the Chamber of Commerce Bills Remaining are the propositions so...

print(ldamodelall.print_topics(num_topics=5, num_words=5))

[(0, u'0.018*"great" + 0.013*"will" + 0.012*"co" + 0.011*"amp" + 0.011*"one"'), (1, u'0.023*"great" + 0.018*"big" + 0.014*"america" + 0.014*"billion" + 0.012*"vote"'), (2, u'0.027*"presid" + 0.019*"will" + 0.018*"trump" + 0.013*"year" + 0.012*"great"'), (3, u'0.020*"will" + 0.013*"china" + 0.013*"peopl" + 0.011*"mueller" + 0.011*"co"'), (4, u'0.020*"look" + 0.016*"will" + 0.011*"presid" + 0.010*"thank" + 0.009*"co"')]


Prints the first 5 topics from the full model.

## What are the topic here?

### Topic 1: great, will, co, amp, one -- Label: ?
### Topic 2: great, big, america, billion, vote -- Label: Election/Voting
### Topic 3: presid, will, trump, year, great -- Label: Tweets about Trump Himself.
### Topic 4: will, china, people, mueller, co -- Label: Sources of Trump's Angst.
### Topic 5: look, will, presid, thank, co -- Label: Tweets about Trump Himself II/Thankfulness. 

## Print out the distribution over topics for a tweet

In [59]:
tweettext[100]

u'I will be in Gulfport and Tupelo, Mississippi, on Monday night doing two Rallies for Senator Hyde-Smith, who has a\u2026 https://t.co/DD7Am2nSiZ'

In [60]:
ldamodelall[corpusall[100]]

[(0, 0.014593961121284617),
 (1, 0.014551081943241893),
 (2, 0.014403287005103337),
 (3, 0.014410350785019746),
 (4, 0.9420413191453505)]

## Breakout Session Exercise

1. Collect and process all tweets from Berkeley's School of Information Account: @BerkeleyISchool.

2. Estimate a 5-topic topic model and label each of the topics.

3. Label one of the tweets using the topic distribution. 